In [6]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [7]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
518,This is the second Baby Burlesk short to be re...,positive
543,i was like watching it right and i was all lik...,positive
183,I really wanted to love this film. I have read...,negative
899,I saw this on a boring Sunday morning just thi...,negative
106,Susan Slept Here turned out to be Dick Powell'...,negative


In [8]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [10]:
import nltk
nltk.download('wordnet')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\upesh\AppData\Roaming\nltk_data...


,review,sentiment
518,second baby burlesk short released probably po...,positive
543,like watching right like oh totally awesome fu...,positive
183,really wanted love film read book daughter lov...,negative
899,saw boring sunday morning morning well drawn f...,negative
106,susan slept turned dick powell swan song perfo...,negative


In [11]:
df['sentiment'].value_counts()

sentiment
negative    265
positive    235
Name: count, dtype: int64

In [12]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [13]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
518,second baby burlesk short released probably po...,1
543,like watching right like oh totally awesome fu...,1
183,really wanted love film read book daughter lov...,0
899,saw boring sunday morning morning well drawn f...,0
106,susan slept turned dick powell swan song perfo...,0


In [14]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Upeshjeengar/MLOps-project.mlflow')
dagshub.init(repo_owner='Upeshjeengar', repo_name='MLOps-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "Upeshjeengar/MLOps-project"

Repository Upeshjeengar/MLOps-project initialized!

<Experiment: artifact_location='mlflow-artifacts:/168389d2cdf548d3a449ecbfe5491ed9', creation_time=1741010158599, experiment_id='0', last_update_time=1741010158599, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [20]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-03 20:48:53,941 - INFO - Starting MLflow run...
2025-03-03 20:49:15,601 - WARNING - Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Upeshjeengar/MLOps-project.mlflow/api/2.0/mlflow/runs/create
2025-03-03 20:49:16,130 - INFO - Logging preprocessing parameters...
2025-03-03 20:49:17,311 - INFO - Initializing Logistic Regression model...
2025-03-03 20:49:17,320 - INFO - Fitting the model...
2025-03-03 20:49:17,396 - INFO - Model training complete.
2025-03-03 20:49:17,396 - INFO - Logging model parameters...
2025-03-03 20:49:17,814 - INFO - Making predictions...
2025-03-03 20:49:17,814 - INFO - Calculating evaluation metrics...
2025-03-03 20:49:17,830 - INFO - Logging evaluation metrics...
2025-03-03 20:49:19,410 - INFO - Saving and logging the model...
2025/03/03 20:49:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `i

🏃 View run legendary-dolphin-53 at: https://dagshub.com/Upeshjeengar/MLOps-project.mlflow/#/experiments/0/runs/7ff532014d154e5bb9d10e10a91d4b55
🧪 View experiment at: https://dagshub.com/Upeshjeengar/MLOps-project.mlflow/#/experiments/0
